In [ ]:
pip install paho-mqtt

In [ ]:
import paho.mqtt.client as mqtt
import time
import folium
import webbrowser
import os
import pandas as pd

In [ ]:
def add_data(dato):
    telemetria = dato.split(',')
    df = pd.read_csv('datos.csv')
    new_data = {'Latitude': float(telemetria[1]), 'Longitude': float(telemetria[2])}
    df = df._append(new_data, ignore_index=True)
    df.to_csv('datos.csv', index=False)

def decode(input_string):
    rssi_hex = input_string[:8]  # First 8 characters for RSSI in hexadecimal
    snr_hex = input_string[8:16]  # Next 8 characters for SNR in hexadecimal
    message_hex = input_string[16:]  # Rest of the characters for the message in hexadecimal

    # Convert hexadecimal strings to integers
    rssi = int(rssi_hex, 16) - 0x100000000
    snr = int(snr_hex, 16)/10

    # Convert the remaining hexadecimal part to ASCII to get the message
    message_ascii = bytes.fromhex(message_hex).decode('utf-8')

    return rssi, snr, message_ascii

def on_message(client, userdata, message):
    #global recibido
    #recibido = True
    print("Message received: " + str(message.payload.decode("utf8")))
    print("Topic: " + str(message.topic))
    rssi, snr, msn = decode(message.payload.decode("utf8"))
    print("RSSI: " + str(rssi))
    print("SNR: " + str(snr))
    print("Message: " + str(msn))
    add_data(str(msn))

def on_connect(client, userdata, flags, rc):
    if rc ==0:
          global connected
          if connected == False:
            print("Connected OK")
            connected = True

datos = pd.DataFrame(columns=['Latitude', 'Longitude'])
datos.to_csv('datos.csv', index=False)

In [ ]:
connected = False
recibido = False

broker_address = "test.mosquitto.org"
port = 1883

client = mqtt.Client("asdasda")
client.connect(broker_address, port = port)
client.on_connect =  on_connect
client.loop_start()
client.subscribe("dragino-2277dc/E66118C4/data")
client.on_message = on_message

while connected!= True:
  time.sleep(20)
while recibido != True:
  time.sleep(0.5)

client.loop_stop()


Connected OK
Message received: ffffffb30000008e312c20362e3236333432392c202d 37352e353631383336
Topic: dragino-2277dc/E66118C4/data
RSSI: -77
SNR: 14.2
Message: 1, 6.263429, -75.561836
Message received: ffffffb30000009d312c20362e3236333431372c202d 37352e353631383434
Topic: dragino-2277dc/E66118C4/data
RSSI: -77
SNR: 15.7
Message: 1, 6.263417, -75.561844
Message received: ffffffb500000096312c20362e3236333433362c202d 37352e353631373938
Topic: dragino-2277dc/E66118C4/data
RSSI: -75
SNR: 15.0
Message: 1, 6.263436, -75.561798
Message received: ffffffb400000091312c20362e3236333435322c202d 37352e353631373938
Topic: dragino-2277dc/E66118C4/data
RSSI: -76
SNR: 14.5
Message: 1, 6.263452, -75.561798
Message received: ffffffaf0000008c312c20362e3236333434342c202d 37352e353631383036
Topic: dragino-2277dc/E66118C4/data
RSSI: -81
SNR: 14.0
Message: 1, 6.263444, -75.561806
Message received: ffffffb200000073312c20362e3236333435372c202d 37352e353631383231
Topic: dragino-2277dc/E66118C4/data
RSSI: -78
SNR:

KeyboardInterrupt: ignored

In [ ]:
# Create a map centered at a specific latitude and longitude
map_center = [6.263432, -75.561852]  # Home
my_map = folium.Map(location=map_center, zoom_start=20)
datos = pd.read_csv('datos.csv')
lat = datos['Latitude'].tolist()
lon = datos['Longitude'].tolist()

for i in range(len(lat)):
    # Extract latitude and longitude values
    print(lat[i],lon[i])
    folium.Marker(location=(lat[i],lon[i]),icon=folium.Icon(color='red', icon='ok-circle', prefix='fa')).add_to(my_map)


# Save the map to an HTML file and open it in a web browser
my_map.save('recorrido.html')


6.263429 -75.561836
6.263417 -75.561844
6.263436 -75.561798
6.263452 -75.561798
6.263444 -75.561806
6.263457 -75.561821
6.263448 -75.561798
6.263474 -75.561798
6.263443 -75.561821
6.263471 -75.561821
6.263461 -75.561943
6.263215 -75.56192


In [ ]:
from IPython.display import IFrame
IFrame(src='recorrido.html', width=900, height=600)